https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations

In [ ]:
import reno

In [ ]:
predator_prey = reno.Model(name="predator_prey", steps=200, doc="Classic predator-prey interaction model example")

with predator_prey:
    # make stocks to monitor the predator/prey populations over time
    rabbits = reno.Stock(init=100.0)
    foxes = reno.Stock(init=100.0)

    # free variables that can quickly be changed to influence equilibrium
    rabbit_growth_rate = reno.Variable(.1, doc="Alpha")
    rabbit_death_rate = reno.Variable(.001, doc="Beta")
    fox_death_rate = reno.Variable(.1, doc="Gamma")
    fox_growth_rate = reno.Variable(.001, doc="Delta")

    # flows that define how much the stocks change in a timestep
    rabbit_births = reno.Flow(rabbit_growth_rate * rabbits)
    rabbit_deaths = reno.Flow(rabbit_death_rate * rabbits * foxes, max=rabbits)
    fox_deaths = reno.Flow(fox_death_rate * foxes, max=foxes)
    fox_births = reno.Flow(fox_growth_rate * rabbits * foxes)

    # hook up inflows/outflows for stocks
    rabbit_births >> rabbits >> rabbit_deaths
    fox_births >> foxes >> fox_deaths

In [ ]:
predator_prey.graph()

In [ ]:
print(predator_prey.get_docs())

In [ ]:
predator_prey(fox_growth_rate=.002, rabbit_death_rate=.002, rabbits_0=120.0)
reno.plot_refs([(rabbits, foxes)])

In [ ]:
with predator_prey:
    minimum_foxes = reno.Metric(foxes.timeseries.series_min())
    maximum_foxes = reno.Metric(foxes.timeseries.series_max())

In [ ]:
trace = predator_prey.pymc(
    n=1000, 
    fox_growth_rate=reno.Normal(.001, .0001), 
    rabbit_growth_rate=reno.Normal(.1, .01),
    observations=[
        reno.Observation(minimum_foxes, 5, [20]),  # likelihood normally distributed around 20 with SD of 5
        reno.Observation(maximum_foxes, 5, [120]), # likelihood normally distributed around 120 with SD of 5
    ]
)

In [ ]:
reno.plot_trace_refs(
    predator_prey, 
    {"prior": trace.prior, "post": trace.posterior}, 
    ref_list=[minimum_foxes, maximum_foxes, fox_growth_rate, rabbit_growth_rate, foxes, rabbits],
    figsize=(8, 5),
)